In [ ]:
from datetime import datetime
import os
import re
import glob
import requests 
import pandas as pd
from bs4 import BeautifulSoup
# regular expression
import re
# numerical analysis
import numpy as np

In [ ]:
# get data
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [ ]:

thead = soup.find_all('thead')[-1]

head = thead.find_all('tr')

tbody = soup.find_all('tbody')[0]

body = tbody.find_all('tr')


In [ ]:
head_rows = []
body_rows = []

for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [ ]:
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         


df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,\nNorth America\n,"82,663,699","+51,959","1,288,275",+287,"54,011,773","+33,526","27,363,651","32,970",,,,,,North America,\n,,,,,
1,,\nAsia\n,"93,093,720","+196,101","1,276,469",+540,"86,334,814","+68,689","5,482,437","25,657",,,,,,Asia,\n,,,,,
2,,\nSouth America\n,"44,679,579","+10,785","1,202,022",+71,"39,724,695","+7,898","3,752,862","14,096",,,,,,South America,\n,,,,,
3,,\nEurope\n,"110,487,545","+468,692","1,587,545","+1,530","84,634,190","+164,299","24,265,810","19,926",,,,,,Europe,\n,,,,,
4,,\nAfrica\n,"10,657,647","+3,070","235,910",+36,"9,441,410","+3,959","980,327","2,905",,,,,,Africa,\n,,,,,


In [ ]:
df_bs.tail(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
221,214,Macao,79,,,,79,,0,,119,,"5,217","7,868","663,062",Asia,"8,393",,127,,,
222,215,Vatican City,29,,,,27,,2,,"36,070",,,,804,Europe,28,,,,,"2,488"
223,216,Samoa,13,,,,3,,10,,65,,,,"200,459",Australia/Oceania,"15,420",,,,,50
224,217,Western Sahara,10,,1,,8,,1,,16,2,,,"620,093",Africa,"62,009","620,093",,,,2
225,218,MS Zaandam,9,,2,,7,,0,,,,,,,,,,,,,


In [ ]:
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)

df_bs = df_bs.drop('#', axis=1)

df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,USA,"70,544,862",,"883,903",,"44,047,799",,"25,613,160","26,332","211,203","2,646","897,864,349","2,688,102","334,014,183",North America,5,378,0
1,India,"38,566,027",,"488,422",,"36,058,806",,"2,018,799","8,944","27,527",349,"711,538,938","507,861","1,401,050,439",Asia,36,"2,869",2
2,Brazil,"23,588,921",,"622,251",,"21,851,922",,"1,114,748","8,318","109,765","2,895","63,776,166","296,766","214,903,595",South America,9,345,3
3,UK,"15,613,283",,"153,202",,"11,850,874",,"3,609,207",675,"228,133","2,239","438,632,905","6,409,078","68,439,318",Europe,4,447,0
4,France,"15,600,647",,"128,114",,"9,844,686",,"5,627,847","3,881","238,186","1,956","216,918,555","3,311,853","65,497,644",Europe,4,511,0


In [ ]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Deathevery X ppl']

df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"334,014,183","70,544,862",,"883,903",,"44,047,799",,"25,613,160","26,332","211,203","2,646","897,864,349","2,688,102"
1,India,Asia,"1,401,050,439","38,566,027",,"488,422",,"36,058,806",,"2,018,799","8,944","27,527",349,"711,538,938","507,861"
2,Brazil,South America,"214,903,595","23,588,921",,"622,251",,"21,851,922",,"1,114,748","8,318","109,765","2,895","63,776,166","296,766"
3,UK,Europe,"68,439,318","15,613,283",,"153,202",,"11,850,874",,"3,609,207",675,"228,133","2,239","438,632,905","6,409,078"
4,France,Europe,"65,497,644","15,600,647",,"128,114",,"9,844,686",,"5,627,847","3,881","238,186","1,956","216,918,555","3,311,853"


In [ ]:
who_region = {}

afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [ ]:
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['Réunion', 'Guadeloupe', 'French Guiana', 'Martinique',
       'French Polynesia', 'Channel Islands', 'Curaçao', 'Mayotte',
       'Aruba', 'Brunei ', 'New Caledonia', 'Isle of Man', 'CAR',
       'Faeroe Islands', 'Cayman Islands', 'Gibraltar', 'Saint Martin',
       'Sint Maarten', 'St. Vincent Grenadines', 'Caribbean Netherlands',
       'British Virgin Islands', 'Turks and Caicos', 'St. Barth',
       'Anguilla', 'Diamond Princess', 'Saint Pierre Miquelon',
       'Wallis and Futuna', 'Montserrat', 'Falkland Islands', 'Macao',
       'MS Zaandam'], dtype=object)

In [ ]:
# fix data
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)

df_bs = df_bs.replace('', 0)

In [ ]:
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,334014183,70544862,0,883903,0,44047799,0,25613160,26332,211203,2646,897864349,2688102,Americas
1,India,Asia,1401050439,38566027,0,488422,0,36058806,0,2018799,8944,27527,349,711538938,507861,South-EastAsia
2,Brazil,South America,214903595,23588921,0,622251,0,21851922,0,1114748,8318,109765,2895,63776166,296766,Americas
3,UK,Europe,68439318,15613283,0,153202,0,11850874,0,3609207,675,228133,2239,438632905,6409078,Europe
4,France,Europe,65497644,15600647,0,128114,0,9844686,0,5627847,3881,238186,1956,216918555,3311853,Europe


In [ ]:
from google.colab import files
from google.colab import drive
import time

#drive.mount('/content/drive')
fileName = time.strftime("%Y-%m-%d-%H%M%S"+".csv")
#df_bs.to_csv('/content/drive/My Drive/'+fileName)

df_bs.to_csv(fileName, index=False)
files.download(fileName)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_bs

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,334014183,70544862,0,883903,0,44047799,0,25613160,26332,211203,2646,897864349,2688102,Americas
1,India,Asia,1401050439,38566027,0,488422,0,36058806,0,2018799,8944,27527,349,711538938,507861,South-EastAsia
2,Brazil,South America,214903595,23588921,0,622251,0,21851922,0,1114748,8318,109765,2895,63776166,296766,Americas
3,UK,Europe,68439318,15613283,0,153202,0,11850874,0,3609207,675,228133,2239,438632905,6409078,Europe
4,France,Europe,65497644,15600647,0,128114,0,9844686,0,5627847,3881,238186,1956,216918555,3311853,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Macao,Asia,663062,79,0,0,0,79,0,0,0,119,0,5217,7868,NaN
214,Vatican City,Europe,804,29,0,0,0,27,0,2,0,36070,0,0,0,Europe
215,Samoa,Australia/Oceania,200459,13,0,0,0,3,0,10,0,65,0,0,0,WesternPacific
216,Western Sahara,Africa,620093,10,0,1,0,8,0,1,0,16,2,0,0,Africa


Analysis

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
import warnings

In [ ]:
countries = df_bs['Country/Region'].unique().tolist()
print("\nTotal countries affected by virus: ",len(countries))
print(countries)


Total countries affected by virus:  218
['USA', 'India', 'Brazil', 'UK', 'France', 'Russia', 'Turkey', 'Italy', 'Spain', 'Germany', 'Argentina', 'Iran', 'Colombia', 'Mexico', 'Poland', 'Indonesia', 'Ukraine', 'Netherlands', 'South Africa', 'Philippines', 'Canada', 'Malaysia', 'Peru', 'Czechia', 'Belgium', 'Thailand', 'Iraq', 'Vietnam', 'Israel', 'Australia', 'Portugal', 'Japan', 'Romania', 'Chile', 'Switzerland', 'Sweden', 'Greece', 'Bangladesh', 'Austria', 'Serbia', 'Hungary', 'Pakistan', 'Denmark', 'Ireland', 'Jordan', 'Kazakhstan', 'Morocco', 'Georgia', 'Cuba', 'Slovakia', 'Nepal', 'Bulgaria', 'Croatia', 'Lebanon', 'UAE', 'Tunisia', 'Bolivia', 'Belarus', 'S. Korea', 'Guatemala', 'Saudi Arabia', 'Costa Rica', 'Azerbaijan', 'Ecuador', 'Panama', 'Sri Lanka', 'Lithuania', 'Slovenia', 'Norway', 'Uruguay', 'Myanmar', 'Dominican Republic', 'Paraguay', 'Kuwait', 'Venezuela', 'Ethiopia', 'Palestine', 'Finland', 'Mongolia', 'Egypt', 'Libya', 'Moldova', 'Honduras', 'Armenia', 'Bosnia and Herz

In [ ]:
df_bs.isna().any()

Country/Region      False
Continent           False
Population          False
TotalCases          False
NewCases            False
TotalDeaths         False
NewDeaths           False
TotalRecovered      False
NewRecovered        False
ActiveCases         False
Serious,Critical    False
Tot Cases/1M pop    False
Deaths/1M pop       False
TotalTests          False
Tests/1M pop        False
WHO Region           True
dtype: bool

In [ ]:
# Replace NaN values by 0
df_bs.fillna(0, inplace=True)
# Remove columns // Country	Confirmed	Deaths	Recovered
df = df_bs
df = df.drop(['Continent','Population','NewCases','NewDeaths', 'NewRecovered', 'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop'], axis=1)
# Rename the columns so that they make sense
df.rename (columns = {'Country/Region':'Country'}, inplace = True)

top = df 

world = top.groupby('Country')['TotalCases','ActiveCases','TotalDeaths', 'TotalRecovered'].sum()
world = top.groupby('Country')['TotalCases','ActiveCases','TotalDeaths', 'TotalRecovered'].sum().reset_index()
world.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Country,TotalCases,ActiveCases,TotalDeaths,TotalRecovered
0,Afghanistan,159516,5991,7390,146135
1,Albania,241512,23593,3286,214633
2,Algeria,230470,67158,6453,156859
3,Andorra,32201,4607,144,27450
4,Angola,95220,7065,1881,86274


In [ ]:
#PLOTTING ON WORLD MAP
figure = px.choropleth(world, locations="Country", locationmode='country names' 
                       , color="ActiveCases",hover_name="Country",range_color=[1,1000],
                      color_continuous_scale="Peach",title="Active Cases in World")
figure.show()

In [ ]:
#PLOTTING ON WORLD MAP
figure = px.choropleth(world, locations="Country", locationmode='country names' 
                       , color="TotalRecovered",hover_name="Country",range_color=[1,1000],
                      color_continuous_scale="Viridis",title="Recovered Cases in World")
figure.show()

In [ ]:
top = top.groupby('Country').sum()
top['ActiveCases'] = top['ActiveCases'].astype(str)
top_active = top.sort_values('ActiveCases', ascending=False).reset_index()
top_active.head(20).style.background_gradient(cmap='Reds')

,Country,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,WHO Region
0,Somalia,24322,1335,13182,9805,EasternMediterranean
1,Libya,403144,5907,387529,9708,EasternMediterranean
2,Singapore,297549,845,287004,9700,WesternPacific
3,Slovenia,579480,5743,477231,96506,Europe
4,Argentina,7576335,118809,6494224,963302,Americas
5,Mauritania,56666,916,46156,9594,Africa
6,Caribbean Netherlands,5946,27,4969,950,0
7,Belize,44959,612,35063,9284,Americas
8,Guadeloupe,94765,762,2250,91753,0
9,Zambia,300587,3885,287646,9056,Africa
